In [29]:
import numpy as np
import pandas as pd

def gaussian_rbf(x, center, width):
    return np.exp(-np.linalg.norm(x - center) ** 2 / (2 * width ** 2))


In [33]:
def treino_RBF(data, num_centers, learning_rate, stop_criterion):
    inputs = data.iloc[:, :-1].values
    outputs = data.iloc[:, -1].values

    num_samples, num_features = inputs.shape
    centers = inputs[np.random.choice(num_samples, num_centers, replace=False)]
    
    output_weights = np.random.rand(num_centers)
    variances = np.ones(num_centers)
    prev_error = float('inf')
    epochs = 0

    while True:
        activations = np.zeros((num_samples, num_centers))
        for i in range(num_samples):
            for j in range(num_centers):
                activations[i, j] = gaussian_rbf(inputs[i], centers[j], width=1.0)

        output = np.dot(activations, output_weights)
        error = np.mean((output - outputs) ** 2)

        if np.abs(prev_error - error) <= stop_criterion:
            break
        
        for j in range(num_centers):
            center_gradient = 0
            variance_gradient = 0
            weight_gradient = 0
            for i in range(num_samples):
                center_gradient += (output[i] - outputs[i]) * output_weights[j] * \
                                   gaussian_rbf(inputs[i], centers[j], width=variances[j]) * \
                                   (inputs[i] - centers[j]) / (variances[j] ** 2)
                variance_gradient += (output[i] - outputs[i]) * output_weights[j] * \
                                     gaussian_rbf(inputs[i], centers[j], width=variances[j]) * \
                                     np.linalg.norm(inputs[i] - centers[j]) ** 2 / (variances[j] ** 3)
                weight_gradient += (output[i] - outputs[i]) * activations[i, j]

            centers[j] -= learning_rate * center_gradient
            variances[j] -= learning_rate * variance_gradient
            output_weights[j] -= learning_rate * weight_gradient

        prev_error = error
        epochs += 1

    final_activations = np.zeros((num_samples, num_centers))
    for i in range(num_samples):
        for j in range(num_centers):
            final_activations[i, j] = gaussian_rbf(inputs[i], centers[j], width=1.0)
            
    final_output = np.dot(final_activations, output_weights)
    final_error = np.sqrt(np.mean((final_output - outputs) ** 2))
    
    for j in range(num_centers):
        print(f"\nCluster {j + 1}:")
        print(f"  Center coordinates: {centers[j]}")
        print(f"  Variance: {variances[j]}")
    
    print(f"\nPesos ajustados (camada escondida): {centers}")
    print(f"Pesos ajustados (camada de saida): {output_weights}")
    print(f"\nNumero de epocas: {epochs}")
    print(f"\nErro Quadrático Médio: {final_error}")

    return centers, variances, output_weights

def teste_RBF(inputs, centers, variances, output_weights):
    num_samples, num_centers = inputs.shape

    activations = np.zeros((num_samples, num_centers))
    for i in range(num_samples):
        for j in range(num_centers):
            activations[i, j] = gaussian_rbf(inputs[i], centers[j], width=variances[j])

    output = np.dot(activations, output_weights)
    return output

In [34]:
data = pd.read_excel('TabelaTreinoFuncoes_RNA_sigaa.xls')
data.head()
data.shape

#parametros

# RBF-1 = 5 neuronios 
# RBF-2 = 10 neuronios
# RBF-3 = 25 neuronios

(150, 4)

### RBF-1 com 5 neurônios

In [35]:
num_centers = 5
learning_rate = 0.01
stop_criterion = 1e-7

# Treino
trained_centers, trained_variances, trained_output_weights = treino_RBF(data, num_centers, learning_rate, stop_criterion)


Cluster 1:
  Center coordinates: [-1.14927066  3.46660889  1.92682836]
  Variance: 0.6161910268987719

Cluster 2:
  Center coordinates: [-0.75605989  0.97363619 -0.40818705]
  Variance: -0.20333927624443318

Cluster 3:
  Center coordinates: [-0.65530465  0.75285587 -2.86935293]
  Variance: 0.13746823921568474

Cluster 4:
  Center coordinates: [1.01161476 1.05956172 1.0205723 ]
  Variance: 2.1676500469469406

Cluster 5:
  Center coordinates: [-0.5222792   2.75863929 -0.71267473]
  Variance: 0.5505734016047878

Pesos ajustados (camada escondida): [[-1.14927066  3.46660889  1.92682836]
 [-0.75605989  0.97363619 -0.40818705]
 [-0.65530465  0.75285587 -2.86935293]
 [ 1.01161476  1.05956172  1.0205723 ]
 [-0.5222792   2.75863929 -0.71267473]]
Pesos ajustados (camada de saida): [-1.6610097  -0.04354975 -2.39633069  1.08645559  0.23275598]

Numero de epocas: 5196

Erro Quadrático Médio: 0.027594893624760083


In [36]:
# Validação
test_data = pd.read_excel('Tabela#6.6_RNA_usp.xls')

# Extrair entradas e saídas do conjunto de teste
test_inputs = test_data.iloc[:, :-1].values
test_outputs = test_data.iloc[:, -1].values

In [37]:
test_data.head()
test_data.shape

(15, 4)

In [38]:
# Testar a rede treinada no conjunto de teste
predictions = teste_RBF(test_inputs, trained_centers, trained_variances, trained_output_weights)

# Calcular Erro Relativo Médio (ERM) em porcentagem
erm_percentage = np.mean(np.abs(predictions - test_outputs) / np.abs(test_outputs)) * 100

# Calcular a variância em porcentagem
variance_percentage = np.var(predictions - test_outputs) / np.var(test_outputs) * 100

print(f"\nErro Relativo Médio (ERM): {erm_percentage}%")
print(f"Variância: {variance_percentage}%")

ValueError: shapes (15,3) and (5,) not aligned: 3 (dim 1) != 5 (dim 0)

### RBF-2 com 10 neurônios

In [39]:
num_centers = 10
learning_rate = 0.01
stop_criterion = 1e-7

# Treino
trained_centers, trained_variances, trained_output_weights = treino_RBF(data, num_centers, learning_rate, stop_criterion)


Cluster 1:
  Center coordinates: [0.3605635  1.11021056 0.22775525]
  Variance: 0.890247700054785

Cluster 2:
  Center coordinates: [0.05059576 0.10801836 0.52952597]
  Variance: 1.018155878908135

Cluster 3:
  Center coordinates: [0.56755131 0.72336545 0.79998424]
  Variance: 0.40370196993767027

Cluster 4:
  Center coordinates: [0.16919656 0.0689022  0.83492975]
  Variance: 0.7938342473052544

Cluster 5:
  Center coordinates: [0.97888662 0.505264   0.34139987]
  Variance: 0.8642970272738222

Cluster 6:
  Center coordinates: [0.56925096 0.8724456  0.04420544]
  Variance: 0.9092115595523463

Cluster 7:
  Center coordinates: [0.94005723 0.10380428 0.66547198]
  Variance: 0.8747576446784556

Cluster 8:
  Center coordinates: [0.65366207 0.3964562  1.00029149]
  Variance: 0.8322327914822419

Cluster 9:
  Center coordinates: [0.54814627 0.89187664 0.29494453]
  Variance: 0.9528134605410133

Cluster 10:
  Center coordinates: [0.1679513  0.05913527 0.43151185]
  Variance: 1.0526851949962726


### RBF-3 com 15 neurônios

In [41]:
num_centers = 15
learning_rate = 0.01
stop_criterion = 1e-7

# Treino
trained_centers, trained_variances, trained_output_weights = treino_RBF(data, num_centers, learning_rate, stop_criterion)


Cluster 1:
  Center coordinates: [0.52967963 0.73419687 0.60140977]
  Variance: 0.9044643826140079

Cluster 2:
  Center coordinates: [0.11646967 0.05633148 0.83515197]
  Variance: 0.8280457110285006

Cluster 3:
  Center coordinates: [ 0.91250188 -0.09904728  0.54229784]
  Variance: 0.8426437139187875

Cluster 4:
  Center coordinates: [1.05248644 0.69605817 0.54141317]
  Variance: 0.8497285346599754

Cluster 5:
  Center coordinates: [ 0.92532445 -0.07460754  0.44973114]
  Variance: 0.8865184407035424

Cluster 6:
  Center coordinates: [1.06503092 0.18754999 0.55509605]
  Variance: 0.911245489526232

Cluster 7:
  Center coordinates: [ 0.20608236 -0.01513395  0.55160267]
  Variance: 0.9426699231672397

Cluster 8:
  Center coordinates: [0.59727482 1.0579169  0.49738103]
  Variance: 0.7266036292220667

Cluster 9:
  Center coordinates: [0.31237066 0.95152695 0.03308198]
  Variance: 0.8905584373221402

Cluster 10:
  Center coordinates: [ 0.50101539  0.8471517  -0.0633241 ]
  Variance: 0.83190